# Overview

**Here, we produce both simple and content-based recommender systems for movies.  The data are obtained from: https://www.kaggle.com/rounakbanik/the-movies-dataset.**

**Simple recommender systems are not personalized to individual users.  In the case of movies, a simple recommender would use movie metadata (e.g., aggregate metadata for each movie, such as average rating and number of ratings) to present movie choices to users.  We formulate a composite score for each movie, but we don't determine similarities of movies.**

***Content-based* recommenders also use movie metadata, but these systems are personalized for each user.  Metadata is used to judge similarities of movies (often this is quantified as a cosine similarity).  Then, if user 1 rated movie A highly, movie A is similar to movie B, and user 1 hasn't yet watched movie B, we would recommend movie B to user 1.**

**Another implementation of a content-based recommender would be to use metadata for users, such as age and sex.  Based on this information, we could determine similarities of users.  Then, if user 1 rated movie A highly, user 1 is similar to user 2, but user 2 hasn't yet watched movie A, we would recommend movie A to user 2.**

**There are ranking algorithms based on *collaborative filtering*, as well.  In this approach, past behavior is used to make recommendations.  For example, if user 1 and user 2 both rated movie A highly, and they both disliked movie B, the two users would be judged to be similar.  Then, if additionally user 2 rated movie C highly, but user 1 hasn't yet consumed movie C, we would recommend movie C to user 1.**

There is also item-based collaborative filtering.  These systems identify similar items, based not on metadata, but on how people have rated them in the past.  For example, if Alice, Bob, and Eve have given 5 stars to The Lord of the Rings and The Hobbit, the system identifies those items as similar.  Therefore, if someone else buys The Lord of the Rings, and hasn't yet consumed The Hobbit, the system would recommend The Hobbit to them.

**TruncatedSVD = PCA (must specify number of components), NMF (must specify number of components), k-Means (must specify number of clusters), LDA**

# Simple recommender system

In [1]:
import pandas as pd

In [3]:
metadata = pd.read_csv('movies_metadata.csv', low_memory=False)

In [4]:
metadata.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


**As mentioned above, we assign a composite score to each movie, based on its metadata.  The formula we use is a weighted average:**

$$\text{Weighted Rating (WR)} = \left(\frac{v}{v+m}\cdot R\right) + \left(\frac{m}{v+m}\cdot C\right),$$

**where:**
- $R$ is the average rating of the movie
- $v$ is the number of ratings received by the movie
- $C$ is the non-weighted average of average ratings across all movies
- $m$ is a threshold, the minimum number of ratings a movie must have received to be considered
- (total number of ratings across all movies)

In [8]:
C = metadata['vote_average'].mean()

print(C)

# C is computed on all movies, prior to filtering out movies with few ratings

5.618207215134185


In [9]:
m = metadata['vote_count'].quantile(0.9)

print(m)

160.0


In [13]:
movies = metadata.loc[metadata['vote_count'] >= m]

print(movies.shape)

(4555, 24)


In [11]:
def weighted_rating(x, m = m, C = C):
    R = x['vote_average']
    v = x['vote_count']
    return (v/(v+m) * R) + (m/(v+m) * C) 

In [15]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
movies['score'] = movies.apply(weighted_rating, axis=1)
# axis=1 ensures that the function is applied row-wise to the dataframe

In [20]:
movies = movies.sort_values('score', ascending = False)

In [21]:
movies[['title', 'score']].head(10)

,title,score
314,The Shawshank Redemption,8.445869
834,The Godfather,8.425439
10309,Dilwale Dulhania Le Jayenge,8.421453
12481,The Dark Knight,8.265477
2843,Fight Club,8.256385
292,Pulp Fiction,8.251406
522,Schindler's List,8.206639
23673,Whiplash,8.205404
5481,Spirited Away,8.196055
2211,Life Is Beautiful,8.187171


In [22]:
movies[['title', 'score']].tail(10)

,title,score
18101,Jack and Jill,4.332366
9651,Alone in the Dark,4.306327
28207,The Boy Next Door,4.303445
24413,Left Behind,4.252002
21238,Sharknado,4.251729
9710,Son of the Mask,4.238168
12911,Disaster Movie,4.082715
3471,Battlefield Earth,3.999793
11557,Epic Movie,3.983225
13566,Dragonball Evolution,3.584903


# Content-based recommender

**We will determine pairwise similarity scores for movies based on their plot descriptions, stored in the "overview" column of metadata.  We start by computing TF-IDF scores for each unique word $t$ (excluding stop words) in each description/document $d$.**

Inverse document frequency, IDF, is obtained by dividing the total number of documents by the number of documents containing the term $t$, and then taking the logarithm of that quotient.  This quotient will always be at least one, so its log will always be nonnegative.

Term frequency, TF, is the raw number of times that term $t$ occurs in document $d$.  One may adjust term frequency for document length, or use a logarithmically scaled frequency similarly to IDF. 

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
# Instantiate a TfidfVectorizer object
# Filter out stop words
# Is it possible to also filter out proper nouns?
# Punctuation is automatically dealt with
tfidf = TfidfVectorizer(stop_words='english')

In [28]:
# Replace missing overviews with empty strings
metadata['overview'] = metadata['overview'].fillna('')

In [29]:
tfidf_matrix = tfidf.fit_transform(metadata['overview'])

**Only took 2.6 seconds to compute all TF-IDF scores**

In [30]:
tfidf_matrix.shape
# 75K+ unique terms (excluding stop words) across all plot descriptions
# Dimensionality reduction?

(45466, 75827)

In [31]:
type(tfidf_matrix) # Most entries are zero, so this matrix is sparse!

scipy.sparse.csr.csr_matrix

**TfidfVectorizer also normalizes the TF-IDF vector for each movie.  Therefore, the cosine of the (smaller) angle between two vectors is obtained simply by taking their dot product.**

In [32]:
from sklearn.metrics.pairwise import linear_kernel
# Without normalization, we would instead import
# from sklearn.metrics.pairwise import cosine_similarity
# which is more computationally intensive

In [33]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [36]:
cosine_sim.shape

(45466, 45466)

**linear_kernel(A,B) just gives the matrix product A B^T**

**We define a function that takes a movie ("movie A") title as input and outputs a list of the 10 most similar movies.  The use case is that if a user likes movie A, then the outputs of the function are recommended to them.**

In [56]:
# Construct a REVERSE map of indices and movie titles
# .drop_duplicates() is probably not necessary
indices = pd.Series(metadata.index, index=metadata['title']).drop_duplicates()

**The contents of the series "indices" are the index values and the index of the series consists of the corresponding titles.  We want to be able to retrieve the index given the title.**

In [45]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Reverse lookup of the index of the movie that matches the title
    idx = indices[title]

    # Get cosine similarity scores for that particular movie with all movies, including itself
    # As a list of tuples, since we need to keep track of indices
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort this list of tuples based on the similarity scores in descending order
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the tuples of the 10 most similar movies, excluding the searched movie itself
    # We are overwriting sim_scores
    sim_scores = sim_scores[1:11]

    # Get just the movie indices, as a list comprehension
    movie_indices = [x[0] for x in sim_scores]

    # Return the top 10 most similar movies
    return metadata['title'].iloc[movie_indices]

In [57]:
get_recommendations('The Dark Knight')

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [53]:
indices['The Dark Knight']

The Dark Knight    12481
The Dark Knight    28700
dtype: int64

In [55]:
metadata.loc[metadata['title'] == 'The Dark Knight']

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
12481,False,"{'id': 263, 'name': 'The Dark Knight Collectio...",185000000,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",http://thedarkknight.warnerbros.com/dvdsite/,155,tt0468569,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,...,2008-07-16,1.004558e+09,152.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Why So Serious?,The Dark Knight,False,8.3,12269.0
28700,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,72003,tt2258647,en,The Dark Knight,In a post-apocalyptic world ravaged by feuding...,...,2011-07-11,0.000000e+00,86.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,The Dark Knight,False,6.3,2.0


**There are two *different* movies with the title "The Dark Knight"...**

**On the other hand:**

In [58]:
get_recommendations('The Dark Knight Rises')

12481                                      The Dark Knight
150                                         Batman Forever
1328                                        Batman Returns
15511                           Batman: Under the Red Hood
585                                                 Batman
21194    Batman Unmasked: The Psychology of the Dark Kn...
9230                    Batman Beyond: Return of the Joker
18035                                     Batman: Year One
19792              Batman: The Dark Knight Returns, Part 1
3095                          Batman: Mask of the Phantasm
Name: title, dtype: object

**We used only the plot descriptions to build this recommender, not other metadata such as genre, name of director, etc.**

# Validation of recommendations

**Two categories:**

- Experimental validation (aka online)
- Non-experimental validation (aka offline)

## Offline metrics

**PRECISION:** fraction of items recommended that are relevant

**RECALL:** fraction of relevant items that are recommended

In some cases (not always, depends on your business!), it’s a fair strategy to recommend only the globally most popular items (a.k.a. bestsellers) to achieve reasonable recall. So here comes the **CATALOG COVERAGE**. Do you wish your users to keep discovering new content to stay loyal? Then you might want to recommend as many different items as possible.

The idea behind **NDCG** is pretty simple. A recommender returns some items and we’d like to compute how good the list is. Each item has a relevance score, usually a non-negative number. That’s gain. (For items we don’t have user feedback for we usually set the gain to zero.)

Now we add up those scores; that’s cumulative gain. We’d prefer to see the most relevant items at the top of the list, therefore before summing the scores we divide each by a growing number (usually a logarithm of the item position) - that’s discounting - and get a DCG. - **CUMULATIVE DISCOUNTED GAIN**

DCGs are not directly comparable between users, so we normalize them. The worst possible DCG when using non-negative relevance scores is zero. To get the best, we arrange all the items in the ideal order, take first K items and compute DCG for them. Then we divide the raw DCG by this ideal DCG to get **NDCG@K**, a number between 0 and 1.

You may have noticed that we denote the length of the recommendations list by K. It is up to the practitioner to choose this number.

The formula for **AVERAGE PRECISION** is:
	$$\sum_{i=1}^K \text{precision at }i\text{ }\times\text{ change in recall at }i$$
    
Then **MEAN AVERAGE PRECISION** is the average of average precision across all users.